<a href="https://colab.research.google.com/github/Wukkkinz-0725/animalImage_classification/blob/master/code_best.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone https://github.com/Wukkkinz-0725/animalImage_classification.git

Cloning into 'animalImage_classification'...
remote: Enumerating objects: 18619, done.
remote: Counting objects: 100% (18619/18619), done.
remote: Compressing objects: 100% (68/68), done.
remote: Total 18619 (delta 18569), reused 18581 (delta 18549), pack-reused 0
Receiving objects: 100% (18619/18619), 13.82 MiB | 22.11 MiB/s, done.
Resolving deltas: 100% (18569/18569), done.


In [2]:
!pip install -q efficientnet_pytorch

  Preparing metadata (setup.py) ... done


In [3]:
import os
import random
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision

from torch.utils.data import Dataset, DataLoader, BatchSampler, random_split
from torchvision import transforms
import torchvision.models as models
from PIL import Image
from sklearn.model_selection import StratifiedShuffleSplit

In [4]:
os.chdir('./animalImage_classification/Released_Data')

## Baseline

In [5]:
# Create Dataset class for multilabel classification
class MultiClassImageDataset(Dataset):
    def __init__(self, ann_df, super_map_df, sub_map_df, img_dir, transform=None):
        self.ann_df = ann_df
        self.super_map_df = super_map_df
        self.sub_map_df = sub_map_df
        self.img_dir = img_dir
        self.transform = transform

    def __len__(self):
        return len(self.ann_df)

    def __getitem__(self, idx):
        img_name = self.ann_df['image'][idx]
        img_path = os.path.join(self.img_dir, img_name)
        image = Image.open(img_path).convert('RGB')

        super_idx = self.ann_df['superclass_index'][idx]
        super_label = self.super_map_df['class'][super_idx]

        sub_idx = self.ann_df['subclass_index'][idx]
        sub_label = self.sub_map_df['class'][sub_idx]

        if self.transform:
            image = self.transform(image)

        return image, super_idx, super_label, sub_idx, sub_label

class MultiClassImageTestDataset(Dataset):
    def __init__(self, super_map_df, sub_map_df, img_dir, transform=None):
        self.super_map_df = super_map_df
        self.sub_map_df = sub_map_df
        self.img_dir = img_dir
        self.transform = transform

    def __len__(self): # Count files in img_dir
        return len([fname for fname in os.listdir(self.img_dir)])

    def __getitem__(self, idx):
        img_name = str(idx) + '.jpg'
        img_path = os.path.join(self.img_dir, img_name)
        image = Image.open(img_path).convert('RGB')

        if self.transform:
            image = self.transform(image)

        return image, img_name

In [6]:
def stratified_split(dataset, stratify_by='superclass_index'):
    from torch.utils.data import Subset
    # Extract labels for stratification
    labels = np.array(dataset.ann_df[stratify_by])

    # Perform stratified split
    sss = StratifiedShuffleSplit(n_splits=1, test_size=0.1)
    train_idx, val_idx = next(sss.split(np.zeros(len(labels)), labels))

    # Create train and validation subsets
    train_subset = Subset(dataset, train_idx)
    val_subset = Subset(dataset, val_idx)

    return train_subset, val_subset

In [7]:
def tensor_to_pil(tensor):
    to_pil_image = transforms.ToPILImage()
    pil_image = to_pil_image(tensor)
    return pil_image

In [8]:
import torchvision.datasets as datasets
import torchvision.transforms as transforms
import random

# Transformations
transform = transforms.Compose([transforms.ToTensor()])

# Load CIFAR-10 and CIFAR-100 datasets
cifar10_dataset = datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
cifar100_dataset = datasets.CIFAR100(root='./data', train=True, download=True, transform=transform)

# Initialize counters and storage
class_counts = {0: 0, 1: 0, 2: 0, 3: 0}  # bird, dog, reptile, novel
filtered_images = []

# Process CIFAR-10 (bird and dog)
for image, label in cifar10_dataset:
    if label in [2, 5]:  # CIFAR-10 labels for bird and dog
        superclass = 0 if label == 2 else 1  # Convert to new superclass label
        if class_counts[superclass] < 2000:
            filtered_images.append((image, superclass, 87))  # subclass 87
            class_counts[superclass] += 1

for image, label in cifar10_dataset:
    if label == 6:  # CIFAR-10 labels for bird and dog
        if class_counts[2] < 1000:
            filtered_images.append((image, 2, 87))  # subclass 87
            class_counts[2] += 1


# Process CIFAR-100 (reptile)
for image, label in cifar100_dataset:
    if 76 <= label <= 80:  # CIFAR-100 label for reptile
        if class_counts[2] < 2000:
            filtered_images.append((image, 2, 87))  # superclass 2, subclass 87
            class_counts[2] += 1

# Add novel images
while class_counts[3] < 3000:
    random_image, label = random.choice(cifar10_dataset)  # Randomly select an image
    if label not in [2, 5, 6]:  # Exclude classes 2, 5, and 6
        filtered_images.append((random_image, 3, 87))  # superclass 3, subclass 87
        class_counts[3] += 1


100%|██████████| 170498071/170498071 [00:11<00:00, 15047989.75it/s]


Extracting ./data/cifar-10-python.tar.gz to ./data


100%|██████████| 169001437/169001437 [00:11<00:00, 15197543.28it/s]


Extracting ./data/cifar-100-python.tar.gz to ./data


In [9]:
filtered_images[0]

(tensor([[[0.6431, 0.4118, 0.4627,  ..., 0.4275, 0.4235, 0.3569],
          [0.6549, 0.4549, 0.2824,  ..., 0.4118, 0.4118, 0.3490],
          [0.5490, 0.5569, 0.4667,  ..., 0.4078, 0.3294, 0.3059],
          ...,
          [0.5451, 0.5569, 0.5294,  ..., 0.3490, 0.3804, 0.4941],
          [0.6392, 0.6000, 0.5725,  ..., 0.3333, 0.3843, 0.4980],
          [0.7176, 0.6902, 0.6039,  ..., 0.3686, 0.3569, 0.4784]],
 
         [[0.8078, 0.5490, 0.5804,  ..., 0.5765, 0.5765, 0.5059],
          [0.8353, 0.6275, 0.4275,  ..., 0.5569, 0.5569, 0.4980],
          [0.7490, 0.7569, 0.6392,  ..., 0.5451, 0.4706, 0.4510],
          ...,
          [0.5804, 0.6078, 0.6118,  ..., 0.5255, 0.5804, 0.6902],
          [0.6157, 0.6431, 0.6431,  ..., 0.5098, 0.5804, 0.6980],
          [0.6000, 0.7137, 0.6039,  ..., 0.5255, 0.5216, 0.6667]],
 
         [[0.3294, 0.2392, 0.3961,  ..., 0.2863, 0.2706, 0.2235],
          [0.3294, 0.1922, 0.1686,  ..., 0.3098, 0.2824, 0.2235],
          [0.2549, 0.2588, 0.3098,  ...,

In [10]:
def show_filtered_image(idx):
    image = filtered_images[idx]
    print(image[1:])
    tensor_to_pil(image[0])

In [11]:
# Read existing data from CSV file or create a new DataFrame if the file doesn't exist
csv_file = 'train_data.csv'
new_csv_file = 'train_data_v1.csv'
existing_df = pd.read_csv(csv_file)
DROP_IDX = [534, 589, 1013, 1231, 1274, 1501, 1827, 1922, 2191, 2195, 2197, 2548, 2575, 2578,
2690, 3049, 3099, 3100, 3292, 3481, 3702, 3743, 4099, 4565, 4850, 4914, 5039, 5150,
5222, 5350, 5557, 5726, 6037, 6262]
existing_df.drop(DROP_IDX, axis=0)
# Prepare CSV data for new images
new_csv_data = {'image': [], 'superclass_index': [], 'subclass_index': []}
for i, filtered_image in enumerate(filtered_images, start=6322):
    file_name = f'{i}.jpg'
    image_path = os.path.join('train_shuffle', file_name)
    image = transforms.ToPILImage()(filtered_image[0])
    image.save(image_path)
    # Update CSV data
    new_csv_data['image'].append(file_name)
    new_csv_data['superclass_index'].append(filtered_image[1])
    new_csv_data['subclass_index'].append(filtered_image[2])

new_df = pd.DataFrame(new_csv_data)
combined_df = existing_df.append(new_df, ignore_index=True)

# Write the combined DataFrame to the CSV file
combined_df.to_csv(new_csv_file, index=False)

<ipython-input-11-3c0448cc6feb>:22: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  combined_df = existing_df.append(new_df, ignore_index=True)


# Data Loader

In [12]:
train_ann_df = pd.read_csv('train_data_v1.csv')
super_map_df = pd.read_csv('superclass_mapping.csv')
sub_map_df = pd.read_csv('subclass_mapping.csv')

train_img_dir = 'train_shuffle'
test_img_dir = 'test_shuffle'

image_preprocessing = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomRotation(10),      # rotate +/- 10 degrees
    transforms.RandomHorizontalFlip(),  # reverse 50% of images
    transforms.RandomVerticalFlip(),    # vertical flip of the image
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.02),  # random color jitter
    transforms.RandomAffine(degrees=0, translate=(0.1, 0.1)),  # random translation
    transforms.RandomResizedCrop(size=(224, 224), scale=(0.8, 1.0)),  # random crop and resize
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])  # normalization
])

# Create train and val split
train_dataset = MultiClassImageDataset(train_ann_df, super_map_df, sub_map_df, train_img_dir, transform=image_preprocessing)
train_dataset, val_dataset = stratified_split(train_dataset)

# Create test dataset
test_dataset = MultiClassImageTestDataset(super_map_df, sub_map_df, test_img_dir, transform=image_preprocessing)

# Create dataloaders
batch_size = 64
train_loader = DataLoader(train_dataset,
                          batch_size=batch_size,
                          shuffle=True,
                          num_workers=4)

val_loader = DataLoader(val_dataset,
                        batch_size=batch_size,
                        shuffle=True,
                        num_workers=4)

test_loader = DataLoader(test_dataset,
                         batch_size=1,
                         shuffle=False,
                         num_workers=4)

In [13]:
def show_image(image_path):
    image = Image.open(image_path).convert('RGB')  # Ensure it's read in RGB format
    return image

# Trainers

In [14]:
class Trainer():
    def __init__(self, model, criterion, optimizer, train_loader, val_loader, test_loader=None, device='cuda'):
        self.model = model
        self.criterion = criterion
        self.optimizer = optimizer
        self.train_loader = train_loader
        self.val_loader = val_loader
        self.test_loader = test_loader
        self.device = device

    def train_epoch(self):
        running_loss = 0.0
        self.model.train()
        for i, data in enumerate(self.train_loader):
            inputs, super_labels, sub_labels = data[0].to(self.device), data[1].to(self.device), data[3].to(self.device)

            self.optimizer.zero_grad()
            super_outputs, sub_outputs = self.model(inputs)
            loss = 0
            loss += self.criterion(super_outputs, super_labels)
            loss += self.criterion(sub_outputs, sub_labels)
            loss.backward()
            self.optimizer.step()

            running_loss += loss.item()

        print(f'Training loss: {running_loss / len(self.train_loader):.3f}')

    def validate_epoch(self):
        super_correct, sub_correct, total = 0, 0, 0
        running_loss = 0.0
        self.model.eval()
        with torch.no_grad():
            for i, data in enumerate(self.val_loader):
                inputs, super_labels, sub_labels = data[0].to(self.device), data[1].to(self.device), data[3].to(self.device)

                super_outputs, sub_outputs = self.model(inputs)
                loss = 0
                loss += self.criterion(super_outputs, super_labels)
                _, super_predicted = torch.max(super_outputs.data, 1)
                super_correct += (super_predicted == super_labels).sum().item()
                loss += self.criterion(sub_outputs, sub_labels)
                _, sub_predicted = torch.max(sub_outputs.data, 1)
                sub_correct += (sub_predicted == sub_labels).sum().item()
                total += super_labels.size(0)
                running_loss += loss.item()

        print(f'Validation Loss: {running_loss / len(self.val_loader):.3f}')
        print(f'Validation Superclass Accuracy: {100 * super_correct / total:.2f} %')
        print(f'Validation Subclass Accuracy: {100 * sub_correct / total:.2f} %')

    def test(self, save_to_csv=False, threshold_super=0.95, threshold_sub=0.95, return_predictions=False):
        self.model.eval()
        if not self.test_loader:
            raise NotImplementedError('test_loader not specified')

        superclass_predictions = {'image': [], 'superclass_index': [], 'superclass_probs': []}
        subclass_predictions = {'image': [], 'subclass_index': [], 'subclass_probs': []}

        with torch.no_grad():
            for i, data in enumerate(self.test_loader):
                inputs, img_name = data[0].to(self.device), data[1]

                super_outputs, sub_outputs = self.model(inputs)

                super_probs = F.softmax(super_outputs, dim=1)
                sub_probs = F.softmax(sub_outputs, dim=1)
                _, super_predicted = torch.max(super_probs, 1)
                _, sub_predicted = torch.max(sub_probs, 1)

                super_predicted[torch.max(super_probs, 1).values < threshold_super] = 3
                sub_predicted[torch.max(sub_probs, 1).values < threshold_sub] = 87

                superclass_predictions['superclass_index'].append(super_predicted.item())
                superclass_predictions['superclass_probs'].append(super_probs.cpu().numpy())
                superclass_predictions['image'].append(img_name[0])

                subclass_predictions['subclass_index'].append(sub_predicted.item())
                subclass_predictions['subclass_probs'].append(sub_probs.cpu().numpy())
                subclass_predictions['image'].append(img_name[0])

        if save_to_csv:
            superclass_df = pd.DataFrame(data=superclass_predictions)
            superclass_df.to_csv('superclass_prediction.csv', index=False)

            subclass_df = pd.DataFrame(data=subclass_predictions)
            subclass_df.to_csv('subclass_prediction.csv', index=False)

        if return_predictions:
            return superclass_predictions, subclass_predictions


In [38]:
class TrainerSuper():
    def __init__(self, model, criterion, optimizer, train_loader, val_loader, test_loader=None, device='cuda'):
        self.model = model
        self.criterion = criterion
        self.optimizer = optimizer
        self.train_loader = train_loader
        self.val_loader = val_loader
        self.test_loader = test_loader
        self.device = device
        self.best_model_wts = None
        self.best_loss = 0

    def train_epoch(self):
        running_loss = 0.0
        self.model.train()
        for i, data in enumerate(self.train_loader):
            inputs, super_labels, sub_labels = data[0].to(self.device), data[1].to(self.device), data[3].to(self.device)

            self.optimizer.zero_grad()
            super_outputs = self.model(inputs)
            loss = 0
            loss += self.criterion(super_outputs, super_labels)
            loss.backward()
            self.optimizer.step()

            running_loss += loss.item()

        print(f'Training loss: {running_loss / len(self.train_loader):.3f}')

    def validate_epoch(self):
        import copy
        super_correct, total = 0, 0
        running_loss = 0.0
        self.model.eval()
        with torch.no_grad():
            for i, data in enumerate(self.val_loader):
                inputs, super_labels, sub_labels = data[0].to(self.device), data[1].to(self.device), data[3].to(self.device)

                super_outputs = self.model(inputs)
                loss = 0
                loss += self.criterion(super_outputs, super_labels)
                _, super_predicted = torch.max(super_outputs.data, 1)
                super_correct += (super_predicted == super_labels).sum().item()
                total += super_labels.size(0)
                running_loss += loss.item()
        if 100 * super_correct / total > self.best_accuracy:
            self.best_model_wts = copy.deepcopy(model.state_dict())
            self.best_accuracy = 100 * super_correct / total
        print(f'Validation Loss: {running_loss / len(self.val_loader):.3f}')
        print(f'Validation Superclass Accuracy: {100 * super_correct / total:.2f} %')

    def test(self, save_to_csv=False, threshold_super=0.95, threshold_sub=0.95, return_predictions=False):
        self.model.eval()
        if not self.test_loader:
            raise NotImplementedError('test_loader not specified')

        superclass_predictions = {'image': [], 'superclass_index': [], 'superclass_probs': []}

        with torch.no_grad():
            for i, data in enumerate(self.test_loader):
                inputs, img_name = data[0].to(self.device), data[1]

                super_outputs = self.model(inputs)

                super_probs = F.softmax(super_outputs, dim=1)
                _, super_predicted = torch.max(super_probs, 1)

                super_predicted[torch.max(super_probs, 1).values < threshold_super] = 3

                superclass_predictions['superclass_index'].append(super_predicted.item())
                superclass_predictions['superclass_probs'].append(super_probs.cpu().numpy())
                superclass_predictions['image'].append(img_name[0])

        if save_to_csv:
            superclass_df = pd.DataFrame(data=superclass_predictions)
            superclass_df.to_csv('superclass_prediction.csv', index=False)

        if return_predictions:
            return superclass_predictions


In [16]:
class AutoEncoderLossTrainer():
    def __init__(self, model, criterion, optimizer, train_loader, val_loader, test_loader=None, device='cuda'):
        self.model = model
        self.criterion = criterion
        self.optimizer = optimizer
        self.train_loader = train_loader
        self.val_loader = val_loader
        self.test_loader = test_loader
        self.device = device

    def custom_loss(self, super_outputs, super_labels, sub_outputs, sub_labels, decoded, original, alpha=0.5):
        classification_loss = nn.CrossEntropyLoss()
        reconstruction_loss = nn.MSELoss()

        loss_super = classification_loss(super_outputs, super_labels)
        loss_sub = classification_loss(sub_outputs, sub_labels)
        loss_recon = reconstruction_loss(decoded, original)

        return alpha * (loss_super + loss_sub) + (1 - alpha) * loss_recon

    def train_epoch(self):
        running_loss = 0.0
        self.model.train()
        for i, data in enumerate(self.train_loader):
            inputs, super_labels, sub_labels = data[0].to(self.device), data[1].to(self.device), data[3].to(self.device)

            self.optimizer.zero_grad()
            super_outputs, sub_outputs, decoded = self.model(inputs)
            loss = self.custom_loss(super_outputs, super_labels, sub_outputs, sub_labels, decoded, inputs)
            loss.backward()
            self.optimizer.step()

            running_loss += loss.item()

        print(f'Training loss: {running_loss / len(self.train_loader):.3f}')

    def validate_epoch(self):
        super_correct, sub_correct, total = 0, 0, 0
        running_loss = 0.0
        self.model.eval()
        with torch.no_grad():
            for i, data in enumerate(self.val_loader):
                inputs, super_labels, sub_labels = data[0].to(self.device), data[1].to(self.device), data[3].to(self.device)

                super_outputs, sub_outputs, decoded = self.model(inputs)
                _, super_predicted = torch.max(super_outputs.data, 1)
                super_correct += (super_predicted == super_labels).sum().item()
                _, sub_predicted = torch.max(sub_outputs.data, 1)
                sub_correct += (sub_predicted == sub_labels).sum().item()
                total += super_labels.size(0)
                loss = self.custom_loss(super_outputs, super_labels, sub_outputs, sub_labels, decoded, inputs)
                running_loss += loss.item()

        print(f'Validation Loss: {running_loss / len(self.val_loader):.3f}')
        print(f'Validation Superclass Accuracy: {100 * super_correct / total:.2f} %')
        print(f'Validation Subclass Accuracy: {100 * sub_correct / total:.2f} %')

    def test(self, save_to_csv=False, threshold_super=0.95, threshold_sub=0.95, return_predictions=False):
        self.model.eval()
        if not self.test_loader:
            raise NotImplementedError('test_loader not specified')

        superclass_predictions = {'image': [], 'superclass_index': [], 'superclass_probs': []}
        subclass_predictions = {'image': [], 'subclass_index': [], 'subclass_probs': []}

        with torch.no_grad():
            for i, data in enumerate(self.test_loader):
                inputs, img_name = data[0].to(self.device), data[1]

                super_outputs, sub_outputs = self.model(inputs)

                super_probs = F.softmax(super_outputs, dim=1)
                sub_probs = F.softmax(sub_outputs, dim=1)
                _, super_predicted = torch.max(super_probs, 1)
                _, sub_predicted = torch.max(sub_probs, 1)

                super_predicted[torch.max(super_probs, 1).values < threshold_super] = 3
                sub_predicted[torch.max(sub_probs, 1).values < threshold_sub] = 87

                superclass_predictions['superclass_index'].append(super_predicted.item())
                superclass_predictions['superclass_probs'].append(super_probs.cpu().numpy())
                superclass_predictions['image'].append(img_name[0])

                subclass_predictions['subclass_index'].append(sub_predicted.item())
                subclass_predictions['subclass_probs'].append(sub_probs.cpu().numpy())
                subclass_predictions['image'].append(img_name[0])

        if save_to_csv:
            superclass_df = pd.DataFrame(data=superclass_predictions)
            superclass_df.to_csv('superclass_prediction_mobilenet_autoencoder.csv', index=False)

            subclass_df = pd.DataFrame(data=subclass_predictions)
            subclass_df.to_csv('subclass_prediction_mobilenet_autoencoder.csv', index=False)

        if return_predictions:
            return superclass_predictions, subclass_predictions

In [17]:
class BCELossTrainer():
    def __init__(self, model, criterion, optimizer, train_loader, val_loader, test_loader=None, device='cuda'):
        self.model = model
        self.criterion = criterion
        self.optimizer = optimizer
        self.train_loader = train_loader
        self.val_loader = val_loader
        self.test_loader = test_loader
        self.device = device

    def custom_loss(self, super_class_output, super_labels, sub_class_output, sub_labels):
        bce_loss = nn.BCEWithLogitsLoss()
        super_class_loss = bce_loss(super_class_output, F.one_hot(super_labels, num_classes=4).float())
        sub_class_loss = bce_loss(sub_class_output, F.one_hot(sub_labels, num_classes=88).float())
        return super_class_loss + sub_class_loss

    def train_epoch(self):
        running_loss = 0.0
        self.model.train()
        for i, data in enumerate(self.train_loader):
            inputs, super_labels, sub_labels = data[0].to(self.device), data[1].to(self.device), data[3].to(self.device)

            self.optimizer.zero_grad()
            super_outputs, sub_outputs = self.model(inputs)
            loss = self.custom_loss(super_outputs, super_labels, sub_outputs, sub_labels)
            loss.backward()
            self.optimizer.step()

            running_loss += loss.item()

        print(f'Training loss: {running_loss / len(self.train_loader):.3f}')

    def validate_epoch(self):
        super_correct, sub_correct, total = 0, 0, 0
        running_loss = 0.0
        self.model.eval()
        with torch.no_grad():
            for i, data in enumerate(self.val_loader):
                inputs, super_labels, sub_labels = data[0].to(self.device), data[1].to(self.device), data[3].to(self.device)

                super_outputs, sub_outputs = self.model(inputs)
                _, super_predicted = torch.max(super_outputs.data, 1)
                super_correct += (super_predicted == super_labels).sum().item()
                _, sub_predicted = torch.max(sub_outputs.data, 1)
                sub_correct += (sub_predicted == sub_labels).sum().item()
                total += super_labels.size(0)
                loss = self.custom_loss(super_outputs, super_labels, sub_outputs, sub_labels)
                running_loss += loss.item()

        print(f'Validation Loss: {running_loss / len(self.val_loader):.3f}')
        print(f'Validation Superclass Accuracy: {100 * super_correct / total:.2f} %')
        print(f'Validation Subclass Accuracy: {100 * sub_correct / total:.2f} %')

    def test(self, save_to_csv=False, threshold_super=0.95, threshold_sub=0.95, return_predictions=False):
        self.model.eval()
        if not self.test_loader:
            raise NotImplementedError('test_loader not specified')

        superclass_predictions = {'image': [], 'superclass_index': [], 'superclass_probs': []}
        subclass_predictions = {'image': [], 'subclass_index': [], 'subclass_probs': []}

        with torch.no_grad():
            for i, data in enumerate(self.test_loader):
                inputs, img_name = data[0].to(self.device), data[1]

                super_outputs, sub_outputs = self.model(inputs)

                super_probs = F.softmax(super_outputs, dim=1)
                sub_probs = F.softmax(sub_outputs, dim=1)
                _, super_predicted = torch.max(super_probs, 1)
                _, sub_predicted = torch.max(sub_probs, 1)

                super_predicted[torch.max(super_probs, 1).values < threshold_super] = 3
                sub_predicted[torch.max(sub_probs, 1).values < threshold_sub] = 87

                superclass_predictions['superclass_index'].append(super_predicted.item())
                superclass_predictions['superclass_probs'].append(super_probs.cpu().numpy())
                superclass_predictions['image'].append(img_name[0])

                subclass_predictions['subclass_index'].append(sub_predicted.item())
                subclass_predictions['subclass_probs'].append(sub_probs.cpu().numpy())
                subclass_predictions['image'].append(img_name[0])

        if save_to_csv:
            superclass_df = pd.DataFrame(data=superclass_predictions)
            superclass_df.to_csv('superclass_prediction.csv', index=False)

            subclass_df = pd.DataFrame(data=subclass_predictions)
            subclass_df.to_csv('subclass_prediction.csv', index=False)

        if return_predictions:
            return superclass_predictions, subclass_predictions

# Models

In [18]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from efficientnet_pytorch import EfficientNet

class CustomEfficientNet(nn.Module):
    def __init__(self, base_model, num_super_classes=4, num_sub_classes=88):
        super(CustomEfficientNet, self).__init__()
        self.base_model = base_model

        in_features = self.base_model._fc.in_features

        self.super_class_classifier = nn.Linear(in_features, num_super_classes)
        self.sub_class_classifier = nn.Linear(in_features, num_sub_classes)

    def forward(self, x):
        features = self.base_model.extract_features(x)

        pooled_features = F.adaptive_avg_pool2d(features, 1).squeeze(-1).squeeze(-1)

        super_class_output = self.super_class_classifier(pooled_features)
        sub_class_output = self.sub_class_classifier(pooled_features)

        return super_class_output, sub_class_output


# Timm Model

In [21]:
!pip install timm
import timm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 10.5 MB/s eta 0:00:00


In [ ]:
class CustomEfficientNetSuper(nn.Module):
    def __init__(self, base_model, num_super_classes=4):
        super(CustomEfficientNetSuper, self).__init__()
        self.base_model = base_model

        in_features = self.base_model._fc.in_features

        self.super_class_classifier = nn.Linear(in_features, num_super_classes)

    def forward(self, x):
        features = self.base_model.extract_features(x)

        pooled_features = F.adaptive_avg_pool2d(features, 1).squeeze(-1).squeeze(-1)

        super_class_output = self.super_class_classifier(pooled_features)

        return super_class_output

In [ ]:
class CustomEfficientNetV2(nn.Module):
    def __init__(self, base_model, num_super_classes=4, num_sub_classes=88):
        super(CustomEfficientNetV2, self).__init__()
        # Load a pretrained EfficientNetV2-S model
        self.base_model = base_model

        # EfficientNetV2 uses a head instead of fc for the classifier
        in_features = self.base_model.classifier[1].in_features

        # Replace the classifier with custom classifiers
        self.super_class_classifier = nn.Linear(in_features, num_super_classes)
        self.sub_class_classifier = nn.Linear(in_features, num_sub_classes)


    def forward(self, x):
        # Extract features from the base model
        features = self.base_model.features(x)
        pooled_features = F.adaptive_avg_pool2d(features, 1).squeeze(-1).squeeze(-1)

        # Classify into super and sub classes
        super_class_output = self.super_class_classifier(pooled_features)
        sub_class_output = self.sub_class_classifier(pooled_features)

        return super_class_output, sub_class_output


In [ ]:
import torch.nn as nn
import torchvision.models as models

class CustomResNet(nn.Module):
    def __init__(self, base_model, num_super_classes=4, num_sub_classes=88):
        super(CustomResNet, self).__init__()
        # Load a pretrained ResNet50 model
        base_model = base_model

        # Replace the final fully connected layer
        in_features = base_model.fc.in_features
        base_model.fc = nn.Identity()  # Remove the original fully connected layer

        self.base_model = base_model
        self.super_class_classifier = nn.Linear(in_features, num_super_classes)
        self.sub_class_classifier = nn.Linear(in_features, num_sub_classes)

    def forward(self, x):
        # Extract features from the base model
        features = self.base_model(x)

        # Classify into super and sub classes
        super_class_output = self.super_class_classifier(features)
        sub_class_output = self.sub_class_classifier(features)

        return super_class_output, sub_class_output

In [ ]:
class MobileNetAutoencoder(nn.Module):
    def __init__(self, num_super_classes=4, num_sub_classes=88):
        super(MobileNetAutoencoder, self).__init__()
        self.mobilenet_features = models.mobilenet_v2(pretrained=True).features

        self.encoder = nn.Sequential(
            self.mobilenet_features,
            nn.AdaptiveAvgPool2d((1, 1))
        )

        self.decoder = nn.Sequential(
            nn.Linear(1280, 512),
            nn.ReLU(),
            nn.Linear(512, 1024),
            nn.ReLU(),
            nn.Linear(1024, 32 * 32 * 3),
            nn.Sigmoid()
        )

        self.super_class_classifier = nn.Linear(1280, num_super_classes)
        self.sub_class_classifier = nn.Linear(1280, num_sub_classes)

    def forward(self, x):
        encoded = self.encoder(x)
        encoded = torch.flatten(encoded, 1)

        decoded = self.decoder(encoded)
        decoded = decoded.view(-1, 3, 32, 32)

        super_class_output = self.super_class_classifier(encoded)
        sub_class_output = self.sub_class_classifier(encoded)

        return super_class_output, sub_class_output, decoded


In [29]:
def test(trainer, save_to_csv=False, super_name='super_pred.csv', sub_name='sub_pred.csv', autocoder=False, return_predictions=False):
    trainer.model.eval()
    if not trainer.test_loader:
        raise NotImplementedError('test_loader not specified')

    superclass_predictions = {'image': [], 'superclass_index': [], 'superclass_probs': []}
    subclass_predictions = {'image': [], 'subclass_index': [], 'subclass_probs': []}

    with torch.no_grad():
        for i, data in enumerate(trainer.test_loader):
            inputs, img_name = data[0].to(trainer.device), data[1]
            if autocoder:
                super_outputs, sub_outputs, _ = trainer.model(inputs)
            else:
                super_outputs, sub_outputs = trainer.model(inputs)
            super_probs = F.softmax(super_outputs, dim=1)
            sub_probs = F.softmax(sub_outputs, dim=1)
            _, super_predicted = torch.max(super_probs, 1)
            _, sub_predicted = torch.max(sub_probs, 1)

            superclass_predictions['superclass_index'].append(super_predicted.item())
            superclass_predictions['superclass_probs'].append(super_probs.cpu().numpy())
            superclass_predictions['image'].append(img_name[0])

            subclass_predictions['subclass_index'].append(sub_predicted.item())
            subclass_predictions['subclass_probs'].append(sub_probs.cpu().numpy())
            subclass_predictions['image'].append(img_name[0])

    if save_to_csv:
        superclass_df = pd.DataFrame(data=superclass_predictions)
        superclass_df.to_csv(super_name, index=False)

        subclass_df = pd.DataFrame(data=subclass_predictions)
        subclass_df.to_csv(sub_name, index=False)

    if return_predictions:
        return superclass_predictions, subclass_predictions


In [30]:
def test_super(trainer, save_to_csv=False, super_name='super_pred.csv', sub_name='sub_pred.csv', autocoder=False, return_predictions=False):
    trainer.model.eval()
    if not trainer.test_loader:
        raise NotImplementedError('test_loader not specified')

    superclass_predictions = {'image': [], 'superclass_index': [], 'superclass_probs': []}

    with torch.no_grad():
        for i, data in enumerate(trainer.test_loader):
            inputs, img_name = data[0].to(trainer.device), data[1]
            super_outputs = trainer.model(inputs)
            super_probs = F.softmax(super_outputs, dim=1)
            _, super_predicted = torch.max(super_probs, 1)

            superclass_predictions['superclass_index'].append(super_predicted.item())
            superclass_predictions['superclass_probs'].append(super_probs.cpu().numpy())
            superclass_predictions['image'].append(img_name[0])

    if save_to_csv:
        superclass_df = pd.DataFrame(data=superclass_predictions)
        superclass_df.to_csv(super_name, index=False)

    if return_predictions:
        return superclass_predictions


In [28]:
# Training loop
def train_model(trainer, epoch):
    for epoch in range(epoch):
        print(f'Epoch {epoch+1}')
        trainer.train_epoch()
        trainer.validate_epoch()
        print('')
    print('Finished Training')

# Experiments

In [47]:
# Init model and trainer
device = 'cuda'
criterion = nn.CrossEntropyLoss()
base_model = timm.create_model('efficientnet_b5', pretrained=True, num_classes=4)
model = base_model.to(device)
optimizer = optim.Adam(model.parameters(), lr=0.001)
trainer_timm_super = TrainerSuper(model, criterion, optimizer, train_loader, val_loader, test_loader)


In [48]:
train_model(trainer_timm_super, epoch=15)

Epoch 1


OutOfMemoryError: ignored

In [40]:
test_super(trainer_timm_super, save_to_csv=True, super_name='super_pred_timm_efb2.csv', autocoder=False, return_predictions=False)

In [ ]:
# Init model and trainer
device = 'cuda'
base_model = EfficientNet.from_pretrained('efficientnet-b5')
model = CustomEfficientNetSuper(base_model).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)
trainer_efb5_super = TrainerSuper(model, criterion, optimizer, train_loader, val_loader, test_loader)

Loaded pretrained weights for efficientnet-b5


In [ ]:
train_model(trainer_efb5_super, epoch=50)

Epoch 1
Training loss: 1.000
Validation Loss: 0.820
Validation Superclass Accuracy: 69.34 %

Epoch 2
Training loss: 0.758
Validation Loss: 0.903
Validation Superclass Accuracy: 72.47 %

Epoch 3
Training loss: 0.666
Validation Loss: 0.638
Validation Superclass Accuracy: 77.43 %

Epoch 4
Training loss: 0.603
Validation Loss: 0.599
Validation Superclass Accuracy: 78.73 %

Epoch 5
Training loss: 0.554
Validation Loss: 0.627
Validation Superclass Accuracy: 75.80 %

Epoch 6
Training loss: 0.536
Validation Loss: 0.548
Validation Superclass Accuracy: 80.17 %

Epoch 7
Training loss: 0.504
Validation Loss: 0.535
Validation Superclass Accuracy: 79.58 %

Epoch 8
Training loss: 0.493
Validation Loss: 0.533
Validation Superclass Accuracy: 79.71 %

Epoch 9
Training loss: 0.461
Validation Loss: 0.538
Validation Superclass Accuracy: 80.10 %

Epoch 10
Training loss: 0.449
Validation Loss: 0.501
Validation Superclass Accuracy: 82.13 %

Epoch 11
Training loss: 0.429
Validation Loss: 0.508
Validation Super

MobileNet_V2

In [ ]:
# Init model and trainer
device = 'cuda'
model = MobileNetAutoencoder().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)
trainer_mbv2 = AutoEncoderLossTrainer(model, criterion, optimizer, train_loader, val_loader, test_loader)

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=MobileNet_V2_Weights.IMAGENET1K_V1`. You can also use `weights=MobileNet_V2_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [ ]:
train_model(trainer_mbv2, epoch=10)

Epoch 1
Training loss: 2.500
Validation Loss: 2.108
Validation Superclass Accuracy: 81.15 %
Validation Subclass Accuracy: 31.56 %

Epoch 2
Training loss: 1.990
Validation Loss: 1.853
Validation Superclass Accuracy: 84.84 %
Validation Subclass Accuracy: 40.57 %

Epoch 3
Training loss: 1.839
Validation Loss: 1.698
Validation Superclass Accuracy: 84.02 %
Validation Subclass Accuracy: 46.31 %

Epoch 4
Training loss: 1.721
Validation Loss: 1.582
Validation Superclass Accuracy: 88.93 %
Validation Subclass Accuracy: 51.78 %

Epoch 5
Training loss: 1.636
Validation Loss: 1.648
Validation Superclass Accuracy: 85.11 %
Validation Subclass Accuracy: 50.68 %

Epoch 6
Training loss: 1.600
Validation Loss: 1.586
Validation Superclass Accuracy: 88.80 %
Validation Subclass Accuracy: 51.78 %

Epoch 7
Training loss: 1.524
Validation Loss: 1.600
Validation Superclass Accuracy: 88.39 %
Validation Subclass Accuracy: 53.01 %

Epoch 8
Training loss: 1.508
Validation Loss: 1.528
Validation Superclass Accuracy:

In [ ]:
test(trainer_mbv2, save_to_csv=True, super_name='super_preds_mbv2.csv', sub_name='sub_preds_mbv2.csv', return_predictions=False)

EffcientNetB7

In [ ]:
# Init model and trainer
device = 'cuda'
base_model = EfficientNet.from_pretrained('efficientnet-b7')
model = CustomEfficientNet(base_model).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)
trainer_efb7 = BCELossTrainer(model, criterion, optimizer, train_loader, val_loader, test_loader)

Loaded pretrained weights for efficientnet-b7


In [ ]:
train_model(trainer_efb7, epoch=10)

Epoch 1
Training loss: 0.524
Validation Loss: 0.774
Validation Superclass Accuracy: 58.74 %
Validation Subclass Accuracy: 9.29 %

Epoch 2
Training loss: 0.271
Validation Loss: 0.319
Validation Superclass Accuracy: 82.79 %
Validation Subclass Accuracy: 17.21 %

Epoch 3
Training loss: 0.227
Validation Loss: 0.219
Validation Superclass Accuracy: 87.43 %
Validation Subclass Accuracy: 19.13 %

Epoch 4
Training loss: 0.202
Validation Loss: 0.257
Validation Superclass Accuracy: 86.07 %
Validation Subclass Accuracy: 20.36 %

Epoch 5
Training loss: 0.177
Validation Loss: 0.182
Validation Superclass Accuracy: 90.30 %
Validation Subclass Accuracy: 24.18 %

Epoch 6
Training loss: 0.169
Validation Loss: 0.199
Validation Superclass Accuracy: 87.98 %
Validation Subclass Accuracy: 19.81 %

Epoch 7
Training loss: 0.164
Validation Loss: 0.192
Validation Superclass Accuracy: 89.21 %
Validation Subclass Accuracy: 23.50 %

Epoch 8
Training loss: 0.166
Validation Loss: 0.327
Validation Superclass Accuracy: 

EfficientNetB5

In [ ]:
# Init model and trainer
device = 'cuda'
base_model = EfficientNet.from_pretrained('efficientnet-b5')
model = CustomEfficientNet(base_model).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)
trainer_efb5 = BCELossTrainer(model, criterion, optimizer, train_loader, val_loader, test_loader)

In [ ]:
train_model(trainer_efb5, epoch=10)

EfficientNetV2

In [ ]:
# Init model and trainer
device = 'cuda'
base_model = models.efficientnet_v2_s(pretrained=True)
model = CustomEfficientNetV2(base_model).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)
trainer_efv2 = BCELossTrainer(model, criterion, optimizer, train_loader, val_loader, test_loader)

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=EfficientNet_V2_S_Weights.IMAGENET1K_V1`. You can also use `weights=EfficientNet_V2_S_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/efficientnet_v2_s-dd5fe13b.pth" to /root/.cache/torch/hub/checkpoints/efficientnet_v2_s-dd5fe13b.pth
100%|██████████| 82.7M/82.7M [00:01<00:00, 63.4MB/s]


In [ ]:
train_model(trainer_efv2, epoch=20)

Epoch 1
Training loss: 0.537
Validation Loss: 0.285
Validation Superclass Accuracy: 81.28 %
Validation Subclass Accuracy: 18.99 %

Epoch 2
Training loss: 0.275
Validation Loss: 0.238
Validation Superclass Accuracy: 84.15 %
Validation Subclass Accuracy: 24.18 %

Epoch 3
Training loss: 0.228
Validation Loss: 0.198
Validation Superclass Accuracy: 87.98 %
Validation Subclass Accuracy: 26.91 %

Epoch 4
Training loss: 0.206
Validation Loss: 0.169
Validation Superclass Accuracy: 90.03 %
Validation Subclass Accuracy: 31.01 %

Epoch 5
Training loss: 0.181
Validation Loss: 0.170
Validation Superclass Accuracy: 89.34 %
Validation Subclass Accuracy: 37.30 %

Epoch 6
Training loss: 0.164
Validation Loss: 0.169
Validation Superclass Accuracy: 90.16 %
Validation Subclass Accuracy: 38.25 %

Epoch 7
Training loss: 0.159
Validation Loss: 0.192
Validation Superclass Accuracy: 88.52 %
Validation Subclass Accuracy: 38.66 %

Epoch 8
Training loss: 0.157
Validation Loss: 0.171
Validation Superclass Accuracy:

EffcientNetV2 CrossEntropyLoss

In [ ]:
# Init model and trainer
device = 'cuda'
base_model = models.efficientnet_v2_s(pretrained=True)
model = CustomEfficientNetV2(base_model).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)
trainer_efv2_celoss = Trainer(model, criterion, optimizer, train_loader, val_loader, test_loader)

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=EfficientNet_V2_S_Weights.IMAGENET1K_V1`. You can also use `weights=EfficientNet_V2_S_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [ ]:
train_model(trainer_efv2_celoss, epoch=30)

Epoch 1
Training loss: 4.430
Validation Loss: 4.317
Validation Superclass Accuracy: 77.46 %
Validation Subclass Accuracy: 25.00 %

Epoch 2
Training loss: 2.974
Validation Loss: 2.708
Validation Superclass Accuracy: 82.24 %
Validation Subclass Accuracy: 37.70 %

Epoch 3
Training loss: 2.414
Validation Loss: 2.162
Validation Superclass Accuracy: 86.34 %
Validation Subclass Accuracy: 50.55 %

Epoch 4
Training loss: 1.970
Validation Loss: 1.823
Validation Superclass Accuracy: 89.89 %
Validation Subclass Accuracy: 56.42 %

Epoch 5
Training loss: 1.756
Validation Loss: 1.937
Validation Superclass Accuracy: 86.34 %
Validation Subclass Accuracy: 56.42 %

Epoch 6
Training loss: 1.531
Validation Loss: 1.706
Validation Superclass Accuracy: 89.62 %
Validation Subclass Accuracy: 58.74 %

Epoch 7
Training loss: 1.575
Validation Loss: 3.264
Validation Superclass Accuracy: 76.91 %
Validation Subclass Accuracy: 35.79 %

Epoch 8
Training loss: 2.045
Validation Loss: 2.156
Validation Superclass Accuracy:

ResNet50

In [ ]:
# Init model and trainer
device = 'cuda'
base_model = models.resnet50(weights="IMAGENET1K_V2")
model = CustomResNet(base_model).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)
trainer_res50 = BCELossTrainer(model, criterion, optimizer, train_loader, val_loader, test_loader)

In [ ]:
train_model(trainer_res50, epoch=30)

Epoch 1
Training loss: 0.461
Validation Loss: 0.331
Validation Superclass Accuracy: 81.01 %
Validation Subclass Accuracy: 28.69 %

Epoch 2
Training loss: 0.240
Validation Loss: 0.247
Validation Superclass Accuracy: 85.52 %
Validation Subclass Accuracy: 33.33 %

Epoch 3
Training loss: 0.212
Validation Loss: 0.214
Validation Superclass Accuracy: 86.75 %
Validation Subclass Accuracy: 39.48 %

Epoch 4
Training loss: 0.195
Validation Loss: 0.178
Validation Superclass Accuracy: 88.66 %
Validation Subclass Accuracy: 38.93 %

Epoch 5
Training loss: 0.175
Validation Loss: 0.180
Validation Superclass Accuracy: 89.62 %
Validation Subclass Accuracy: 43.31 %

Epoch 6
Training loss: 0.167
Validation Loss: 0.154
Validation Superclass Accuracy: 91.53 %
Validation Subclass Accuracy: 43.85 %

Epoch 7
Training loss: 0.156
Validation Loss: 0.193
Validation Superclass Accuracy: 87.98 %
Validation Subclass Accuracy: 45.63 %

Epoch 8
Training loss: 0.156
Validation Loss: 0.191
Validation Superclass Accuracy:

In [ ]:
# Init model and trainer
device = 'cuda'
base_model = models.resnet18(weights="IMAGENET1K_V1")
model = CustomResNet(base_model).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)
trainer_res18 = BCELossTrainer(model, criterion, optimizer, train_loader, val_loader, test_loader)

Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth
100%|██████████| 44.7M/44.7M [00:00<00:00, 180MB/s]


In [ ]:
train_model(trainer_res18, epoch=20)

Epoch 1
Training loss: 0.387
Validation Loss: 0.297
Validation Superclass Accuracy: 78.28 %
Validation Subclass Accuracy: 18.31 %

Epoch 2
Training loss: 0.271
Validation Loss: 0.265
Validation Superclass Accuracy: 81.42 %
Validation Subclass Accuracy: 20.77 %

Epoch 3
Training loss: 0.249
Validation Loss: 0.267
Validation Superclass Accuracy: 82.10 %
Validation Subclass Accuracy: 20.77 %

Epoch 4
Training loss: 0.228
Validation Loss: 0.240
Validation Superclass Accuracy: 85.11 %
Validation Subclass Accuracy: 22.54 %

Epoch 5
Training loss: 0.216
Validation Loss: 0.249
Validation Superclass Accuracy: 84.43 %
Validation Subclass Accuracy: 21.99 %

Epoch 6
Training loss: 0.212
Validation Loss: 0.220
Validation Superclass Accuracy: 85.93 %
Validation Subclass Accuracy: 23.36 %

Epoch 7
Training loss: 0.196
Validation Loss: 0.225
Validation Superclass Accuracy: 85.25 %
Validation Subclass Accuracy: 24.86 %

Epoch 8
Training loss: 0.191
Validation Loss: 0.193
Validation Superclass Accuracy:

# Predict Test Data

In [ ]:
test(trainer_mbv2, save_to_csv=True, super_name='super_preds_mbv2.csv', sub_name='sub_preds_mbv2.csv', autocoder=True, return_predictions=False)

In [ ]:
test(trainer_efb5, save_to_csv=True, super_name='super_preds_efb5.csv', sub_name='sub_preds_efb5.csv', return_predictions=False)

In [ ]:
test(trainer_efb7, save_to_csv=True, super_name='super_preds_efb7.csv', sub_name='sub_preds_efb7.csv', return_predictions=False)

In [ ]:
test(trainer_res18, save_to_csv=True, super_name='super_preds_res18.csv', sub_name='sub_preds_res18.csv', return_predictions=False)

In [ ]:
test(trainer_res50, save_to_csv=True, super_name='super_preds_res50.csv', sub_name='sub_preds_res50.csv', return_predictions=False)

In [ ]:
test(trainer_efv2, save_to_csv=True, super_name='super_preds_efv2.csv', sub_name='sub_preds_efv2.csv', return_predictions=False)

In [ ]:
test(trainer_efv2_celoss, save_to_csv=True, super_name='super_preds_efv2celoss.csv', sub_name='sub_preds_efv2celoss.csv', return_predictions=False)

In [ ]:
def transform_df(df, threshold):
    df = df.copy()
    df['probs'] = df['subclass_probs']
    df['probs'] = df['probs'].str.strip('[]').str.split()
    df['probs'] = df['probs'].apply(lambda x: [float(i) for i in x[:-1]])
    df['Max_Prob'] = df['probs'].apply(max)
    df['Target'] = df['probs'].apply(lambda x: x.index(max(x)))
    df['Target'] = df['Target'].where(df['Max_Prob'] > threshold, 87)
    # print distribution
    print(df['Target'].value_counts())
    return df

In [ ]:
def output_df(df, output_name):
    output = pd.DataFrame({'ID': df['image'], 'Target': df['Target']})
    output.to_csv(output_name, index=False)
    return output

EffcientNetB7 BCE more data

In [ ]:
filename='subclass_prediction.csv'
df = pd.read_csv(filename)
sub_df_transformed = transform_df(df, 0.3)
output_df(sub_df_transformed, 'sub_test_effcientNet_moreData.csv')

87    11816
37      262
50      136
71       66
41       43
18       22
24       17
28       12
35        2
75        1
Name: Target, dtype: int64


,ID,Target
0,0.jpg,87
1,1.jpg,87
2,2.jpg,87
3,3.jpg,87
4,4.jpg,37
...,...,...
12372,12372.jpg,87
12373,12373.jpg,87
12374,12374.jpg,87
12375,12375.jpg,87


In [ ]:
sub_df_transformed = transform_df(df, 0.8)
output_df(sub_df_transformed, 'sub_test_effcientNet_moreData.csv')

4     2728
6     1420
24    1073
2      916
57     746
37     695
21     669
50     522
75     476
71     473
30     435
65     347
62     323
72     273
52     262
68     228
41     167
26     118
36     113
51      78
28      74
43      45
7       34
18      30
77      26
70      26
12      15
25      14
9       12
64      12
35       8
84       5
13       5
63       2
78       2
34       1
38       1
49       1
17       1
15       1
Name: Target, dtype: int64


,ID,Target
0,0.jpg,21
1,1.jpg,24
2,2.jpg,71
3,3.jpg,4
4,4.jpg,37
...,...,...
12372,12372.jpg,4
12373,12373.jpg,71
12374,12374.jpg,57
12375,12375.jpg,52
